# Documentation
## CS207 Final Project, Group 28
#### Team Members:
* Josh Bodner  
* Théo Guenais  
* Daiki Ina  
* Junzhi Gong

# Introduction

Ever since the notion of a derivative was first defined in the days of Newton and Leibniz, differentiation has become a crucial component across quantitative analyses. Today, differentiation is applied within a wide range of scientific disciplines from cell biology to electrical engineering and astrophysics. Differentiation is also central to the domain of optimization where it is applied to find maximum and minimum values of functions.

Because of the widespread use and applicability of differentiation, it would be highly beneficial if scientists could efficiently calculate derivatives of functions using a Python package. Such a package could save scientists time and energy from having to compute derivatives symbolically, which can be especially complicated if the function of interest has vector inputs and outputs.

Our Python package, **autodiff**, addresses this need by allowing the user to implement the forward mode of automatic differentiation (AD). Using AD, we are able to calculate derivatives to machine precision in a manner that is less costly than symbolic differentiation.

# Background
In this section we provide a brief overview of the mathematical concepts relevant to our implementation:

##### 1. Multivariable Differental Calculus:

Let $f: {\rm R^m} \rightarrow {\rm R^n}: x \mapsto f(x)$ . Under certain regularity and smoothness assumptions, we define the derivative of f as $ f': {\rm R^m} \rightarrow L_{{\rm R^m}, {\rm R^n}}$, with $ L_{{\rm R^m}, {\rm R^n}}$ being the space of the linear mapping $ {\rm R^m}\rightarrow {\rm R^n}$ which can be transformed to $ {\rm R^{mxn}}$.

*This general definition might be broken due to lack of smoothness, in which case, we refer to "directional derivatives" and specifically "Gateaux-derivatives".*

From this lense, we can understand the derivative of a function evaluated at a given point to be a matrix. When this function is real-valued, we can obtain a vector $ \nabla_x f \in {\rm R^m} $.

Specifically, the gradient of a scalar-valued multivariable function is a vector of its partial derivatives with respect to each input:

$\nabla f = 
  \begin{bmatrix}
    \frac{\partial f}{\partial x_1} \\
    \frac{\partial f}{\partial x_2} \\
    \frac{\partial f}{\partial x_3} \\
    \vdots
  \end{bmatrix}$

##### 2. Chain Rule
Let $ f: {\rm R^m} \rightarrow {\rm R^n} $ and $ g: {\rm R^p} \rightarrow {\rm R^m} $. 
Then $ f \circ g: {\rm R^p} \rightarrow {\rm R^n} $ is such that (under regularity assumptions), $ (f \circ g)'(x) = f'(g(x)) \cdot g'(x) $. This operational rule, known as the chain rule, is a crucial part of AD.

---

*Example:*  
$ f(x) = sin(x)$  
$ g(x) = e^{x}$  
$ (f \circ g)(x) = sin(e^{x})$  
 
$(f \circ g)'(x) = f'(g(x))g'(x) = cos(e^{x})e^{x}$

##### 3. Automatic Differentiation

*Automatic Differentiation and Forward Mode:* 

Automatic differentiation is a process for evaluating derivatives computationally at a particular evaluation point. Specifically, the forward mode of automatic differentiation calculates the product of the gradient with a seed vector $ \nabla f \cdot p $ or the product of the Jacobian with a seed vector ($Jp$) if $f$ is a multi-dimensional function.

In the forward mode of automatic differentiation, the calculation is done in steps where the partial derivatives and values are computed at each step of the computational graph. The edges of the computational graph are such that transitioning from node to node involves simple operations or calculation of elementary functions such as $sin(x)$, $cos(x)$, $e^{x}$, etc.

*Precision and Efficiency:*

Automatic differentiation is able to evaluate derivatives to machine precision. This is because it computes exact derivatives using the component elementary functions at each step of the computational graph. The precision of automatic differentiation is one of its advantages over other methods such as numerical differentiation, which can suffer from floating point precision errors.

The forward mode of automatic differentiation is also particularly efficient when the the number of functions to evaluate is much larger than the number of independent variables $ n >> m$  for $f: {\rm R^m} \rightarrow {\rm R^n}$. This is because forward mode involves performing one sweep over the computational graph for each of the m independent variables. In comparison, the reverse mode of automatic differentiation requires n sweeps over the computational graph and is thus more efficient when $m >> n$.

---
*Graph Structure Example:*  

$ y = sin(e^{x}) + x$  
Calculate $\frac{dy}{dx}$ at x=5 

Evaluation trace of forward mode of AD: 

| Trace | Elementary Function | Current Value |  Derivative          | Derivative Value |
|-------|---------------------|---------------|----------------------|------------------|
| $x_1$ |        $x_1$        |    5          | $\dot{x_1}$          |        1         |
| $x_2$ |        $e^{x_1}$    |    148.413    | $e^{x_1}\dot{x_1}$   |        148.413   |
| $x_3$ |        $sin(x_2)$   |    0.524      | $cos(x_2)\dot{x_2}$  |       -126.425   |
| $x_4$ |        $x_3 + x_1$  |    5.524      | $\dot{x_3}+\dot{x_1}$|       -125.425   |


The computational graph can be seen below:  
![example_graph](figs/milestone1_graph.png "Computational Graph")

# How to use autodiff

**Install autodiff package:**

----
*Option 1:*

Our package is available on PyPI. The preferred way for installing our package via PyPI is as follows:

First, the user can (optionally) create a virtual environment before installing our package dependencies. A virtual enviroment would allow the user to compartmentalize the the dependencies for our package from the dependencies of other projects they might be working on.

For users unfamiliar with virtual environments, the tool virtualenv can be easily installed via:

    sudo easy_install virtualenv

The user can then activate a virtual environment via:

    virtualenv env
    source env/bin/activate
    
Note: the user can later deactivate the virtual enviroment via:

    deactivate
    
Then, the user can simply install our package from PyPI via:

    pip install autodiff-ADdictedtoCS

----
*Option 2:*

Alternatively, the autodiff package can be downloaded directly from our organization's github repository at: https://github.com/ADdictedtoCS/cs207-FinalProject.git

The user can then install the requirements via:

    pip install -r requirements.txt
    
----

The user can then import our package as follows...   

In [1]:
#  Import statements
import autodiff
import autodiff.function as F
import autodiff.optim as optim
from autodiff.variable import Variable

See below for an example of how the user can interact with our package:

1. Create a "variable" instantiated with a vector of initial values
    - For a multivariate function of 3 inputs for example, "variable" takes in the initial value
      of each input
2. Define a function 
    - Functions take variables as inputs and return new variables with updated values and gradients
3. Call the function and get the value and gradient of the resulting variable 
    - The user can simply print the variable or alternatively call .val or .grad on the variable

*Remarks*: 
- The user can benefit from a natural and simple way to interact with the Variables and Functions.
-  autodiff.function also supports a range of different elementary functions. See "Implementation Details" section below for further details.



### Example 1: Single input, single output

In [2]:
# Example R^1 -> R^1

# Define a variable with an initial value
x = Variable(0.)
print("Input x:")
print(x)

# Define a function
def my_func(x):
    return F.sin(F.exp(x))

# Variable z is the result of calling function on x
z = my_func(x)

# Get value and gradient of z
print("\nOutput z:")
print(z)

# Alternatively:
print('The value is: {}'.format(z.val))
print('The gradient is: {}'.format(z.grad))

Input x:
Value: 0.0
Gradient: 1.0

Output z:
Value: 0.8414709848078965
Gradient: 0.5403023058681398
The value is: 0.8414709848078965
The gradient is: 0.5403023058681398


Above, we initialize a variable x with a value of 0

We then define the function:
𝑓(𝑥)=𝑠𝑖𝑛(𝑒𝑥𝑝(𝑥)) 
Lastly, we get the value and the gradient of the function at the initial point:
Value:  𝑠𝑖𝑛(𝑒𝑥𝑝(0))=0.841 
Gradient:  𝑒𝑥𝑝(0)∗𝑐𝑜𝑠(𝑒𝑥𝑝(0))=0.540

### Example 2: Multi input, single output

In [3]:
# Example R^3 -> R^1

# Define a variable with a vector of initial values
X = Variable([0,1,2])
print("Input X")
print(X)

# Define a function
# Unroll allows us to define an expression using the individual input variables
def my_func(X):
    x, y, z = X.unroll()
    return F.cos(x) + y * z

# Variable out is the result of calling function on X
out = my_func(X)

# Print value and gradient of the output
print('\nOutput:')
print('The value is: {}'.format(out.val))
print('The gradient is: {}'.format(out.grad))

Input X
Value: [[0.]
 [1.]
 [2.]]
Gradient: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Output:
The value is: 3.0
The gradient is: [[0. 2. 1.]]


Above, we initialize "Variable" X with a vector of initial values [0, 1, 2]

X is thus a vector of 3 input variables, which can be obtained via "unroll".
We can call these input variables x, y, z

We then define a function in terms of the input variables  
     $f(x,y,z) = cos(x) + y*z$

Lastly, we get the value and the gradient of the function at the initial point:  
Value:  
$cos(0) + 1*2 = 3$  
Gradient:  
$ \frac{\partial{f}}{\partial{x}} = -sin(x) = 0$  
$ \frac{\partial{f}}{\partial{y}} = z = 2 $  
$ \frac{\partial{f}}{\partial{z}} = y = 1 $  

### Case 3: Multi input, multi output

In [4]:
# Example R^3 -> R^3

# Define a variable with a vector of initial values
X = Variable([0,1,2])
print("Input X")
print(X)

# Define a function
# Unroll allows us to define an expression using the individual input variables
# Concat is used to build the output vector of the function
def my_func3(X):
    x, y, z = X.unroll()
    o1 = F.sin(F.exp(x))
    o2 = y + F.cosh(z)
    o3 = F.Log(10)(z)
    out_X = F.concat([o1, o2, o3])
    return out_X

# Variable out is the result of calling function on X
out = my_func3(X)

# Print value and gradient of the output
print('\nOutput:')
print('The value is: \n{}'.format(out.val))
print('The gradient is: \n{}'.format(out.grad))

Input X
Value: [[0.]
 [1.]
 [2.]]
Gradient: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Output:
The value is: 
[[0.84147098]
 [4.76219569]
 [0.30103   ]]
The gradient is: 
[[0.54030231 0.         0.        ]
 [0.         1.         3.62686041]
 [0.         0.         0.21714724]]


Additionally, users of our package may be interested in appyling it to solve optimization problems. As such, we provide an example of how our package could be used to implement the Newton-Raphson method for root finding:

In [5]:
def newtons_method(function, guess, epsilon):
    x = Variable(guess)
    f = function(x)
    i = 0
    max_out = False
    while abs(f.val) >= epsilon and max_out == False:
        x = x - f.val / f.grad
        f = function(x)
        print('Current x: {}'.format(x.val))
        i += 1
        if i >= 10000:
            max_out = True
    print('The root of the function is: {}'.format(x.val))
            

def my_func(x):
    return 5*(x-2)**3

guess = 5
epsilon = 0.000001

newtons_method(my_func, guess, epsilon)

Current x: 4.0
Current x: 3.3333333333333335
Current x: 2.888888888888889
Current x: 2.5925925925925926
Current x: 2.3950617283950617
Current x: 2.263374485596708
Current x: 2.1755829903978054
Current x: 2.11705532693187
Current x: 2.078036884621247
Current x: 2.052024589747498
Current x: 2.034683059831665
Current x: 2.023122039887777
Current x: 2.015414693258518
Current x: 2.0102764621723455
Current x: 2.0068509747815635
Current x: 2.0045673165210425
The root of the function is: 2.0045673165210425


# Software Organization

##### Directory Structure:

We have our main implementation stored in the autodiff directory. This is where modules for our implementation of the forward mode of AD are located. The autodiff directory has a subdirectory containing tests.

```
cs207-FinalProject/
    docs/
        documentation.ipynb
        milestone1.ipynb
        milestone2.ipynb     
        demo/
            demo.py
            demo_newtons_method.py
    autodiff/
        __init__.py
        function.py
        optim.py
        variable.py
        utils.py     
        tests/
            __init__.py
            test_function.py
            test_reverse_function.py
            test_reverse_variable.py
            test_optim.py
            test_variable.py
            test_utils.py
    demo.ipynb
    README.md
    requirements.txt
    
```
##### Basic Modules:
We included four modules in the autodiff directory as shown above. 
The function module contains our function class as well as the implementation of our elementary functions that constitute the computational graph. The variable module has our Variable and ReverseVariable class as well as the implementation of all the basic operations (addition, multiplication, subtraction, division, power). The Varibale class serves as the information flow within the computational graph for forward mode, and the ReverseVariable class accomplishes the same purpose for reverse mode. The utils module contains functions to be called by the variable class to make sure the input is of the correct type for the definition of a variable. The optim module has a superclass Optimizer, and its subclasses GradientDescent, RMSProp and Adam. They use automatic differentiations to find local critical points.

##### Testing:

Our test suite lives within the autodiff directory (see above). Additionally for continuous integration and code coverage we utilize both TravisCI and CodeCov. We set up basic functionality for TravisCI and CodeCov for our repository.

Tests can be run simply via:  
    
    pytest
    
(or optional-for a more detailed coverage)

    pytest --doctest-modules --cov --cov-report term-missing

##### Distribution and Packaging:

We distributed our package using PyPI (and upload with Twine). The user is able to install our package via the command line: "pip install autodiff-ADdictedtoCS"

Optionally, the user can manually install our package by downloading our github repo from https://github.com/ADdictedtoCS/cs207-FinalProject.git , setting the current directory to the directory containing requirements.txt, and running "pip install -r requirements.txt". Before doing this, the user may want to set up a virtual enviroment using virtualenv. 

For more details on precisely how the user can install our package, along with examples of how to use our package, please refer to the *"How to use autodiff"* section above.

# Implementation Details


##### Core Data Structures:

For each function defined by users, we create a Directed Acyclic Graph (DAG) data structure, which contains all computing nodes for the forward mode of AD.

Our operation is based upon two main concepts: 
* **variable** carries the information flow through the computational graph. (see below).
* **function** implements the elementary functions and constitutes the edges (or nodes, depending on ones's interpretation of the computational graph).

Within each node, we compute the value and gradient of the function at that point in the computational graph. Therefore, the lists from all nodes form the computation table in the forward mode of AD.

##### Classes:

**Variable**  
Our core class, which carries the information flow within the computational graph.

Variables are initialized at a particular value passed in by the user, and have a default gradient of 1 (the "seed vector").

All of the basic functionality for algebra involving variables is implemented within the Variable class through operator overloading.

For example, we define what addition means for Variables in terms of their values and gradients here:

```Python
    def __add__(self, other):
        if isinstance(other, Variable):
            out_val = self.val + other.val
            out_grad = self.grad + other.grad
            return Variable(val=out_val, grad=out_grad)
        else:
            new_val = get_right_shape(other)
            out_val = self.val + get_right_shape(other)
            out_grad = self.grad
            return Variable(val=out_val, grad=out_grad)
```

In the current implementation, Variable can hold multiple values and gradient values (derivatives of multiple points). We rely on numpy arrays for this functionality to allow Variable hold an array of values and an array for gradient as well. 


**Function**  
When called on a variable, a function returns a new variable with a transformed value and gradient.

A function has the methods *get_val* and *get_grad* that denote how the function transforms a variables val and grad. Note that these methods are not implemented for the base class but for the elementary functions which are subclasses of function (see below for more details).

The function class is also where we implement the chain rule for calculating gradients:

```Python

    def __call__(self, x):
        """
        Implements the chain rule.
        Input: autodiff.Variable type holding a val and grad
        Output:  autodiff.Variable type holding the val, grad of the
        transformed variable
        """
        out_val = self.get_val(x.val)
        out_grad = np.dot(self.get_grad(x.val), x.grad)
        return Variable(val=out_val, grad=out_grad)

```


##### Important Attributes:

In our current implementation, the key attributes for the **Variable** class are:

*val* = The value. Can be an int, float, or numpy.array. When a variable is first initialized, this is passed in by the user.

*grad* = The gradient. Can be an int, float, or numpy.array. When the variable is first initialized this has a default value of 1 (the "seed vector").

As mentioned above, functions take in a variable and output a new variable with a transformed value and gradient.

The value and gradient of a variable can be obtained by the user by calling .val or .grad on the variable. Additionally, the user can print a variable to see its value and gradient.

```Python
    # Define a variable with an initial value
    x = Variable(5.)
    
    x.val
    >>>> [5.]
    
    x.grad
    >>>> [1.]
    
    print(x)
    >>>> Value: [5.]
    >>>> Gradient: [1.]

```


#### External dependencies:
External dependencies for our package are listed in requirements.txt

The required external dependencies are Numpy and Matplotlib.

Required:
- Numpy 1.16.4 (for vector operations)
- Matplotlib 3.1.1 (for visualization)


#### Elementary functions:

The elementary functions such as **sin**, **exp**, etc are all implemented as subclasses of **Function**.

For each elementary function, we explicitedly define its value and derivative. For example, $e^x$ below:

```Python
    class Exponent(Function):
        """Exponential"""    
        def get_val(self, x):
            return np.exp(x)
    
        def get_grad(self, x):
            return np.exp(x)
```

In our current implementation, the elementary functions we have implemented along with their derivatives are as follows:

| Elementary Function      |  Derivative            |
|--------------------------|------------------------|
|        $e^x$             |   $e^x$                |
|        $sin(x)$          |   $cos(x)$             |
|        $cos(x)$          |   $-sin(x)$            |
|        $tan(x)$          |   $\frac{1}{cos(x)^2}$ | 
|        $arcsin(x)$       |   $\frac{1}{\sqrt{1-x^2}}$ | 
|        $arccos(x)$       |   $-\frac{1}{\sqrt{1-x^2}}$ | 
|        $arctan(x)$       |   $\frac{1}{1+x^2}$ | 
|        $log(x)$          |   $\frac{1}{x}$ | 
|        $logistic(x)$     |   $x(1-x)$ | 
|        $sqrt(x)$         |   $\frac{1}{2\sqrt{x}}$ | 
 


# Our Extention

##### Vector Valued and Multivariate Functions:

Our immediate next step is to update our package to allow for functions with multiple inputs and outputs. We can accomplish this by reworking our variable class to allow for an array of inputs and to compute the full gradient at each step of the computational graph.

Thus, we will also have to update our code to incorporate matrix algebra, for which we plan to rely on the numpy package.

Implementing this change will involve the following:

a. The initialization of Variable objects will also accept vector-based values, which could be built using the numpy library.  

b. The attributes of Variable class, "val" and "grad", will be vector valued as well.  

c. Operators such as "+", "-", "\*", and "/" will need to be redefined. Each operator will now involve performing vector operations. We foresee a primary challenge being implementing vector operations for "/" and "\*\*", as we will have to be extra careful to check for edge-cases such as division by 0.  

d. For the "\*" operator, additional work will also need to be done. This operator will need to accept matrix values, and check whether two matrices are of the correct shapes to be multiplied.  

e. We will also need to rework the elementary functions to allow for vector-valued inputs.  

f. The definition of "grad" will change. In the forward mode of autodiff, a variable will record the gradient for each "root variable" (variables we create manually). Therefore, we can change "grad" as a **dict** of gradients, with one gradient value for each root variable. 

g. The definition of operators will change. When creating a new variable by performing an operation between two variables, we should calculate the gradient for each root variable. If the gradient of one root variable does not exist for one variable, then it will be 0 for that variable. 


##### Argument Parser:

An additional feature we would like to implement that we believe could improve ease of use of our package for users is an argument parser, which would allow users to input functions as strings.

In our current implementation, as shown in the "How to use autodiff" section, the user must enter functions by specifically calling elementary functions such as:

    F.sin(F.exp(x))
    
Our argument parser would be built so that users could have the option of entering functions more naturally as strings such as:

    "sin(e^x)"
   
We would use regular expressions to implement our argument parser in a way that is robust and accounts for a wide range of possible user inputs.

##### Reverse Mode:

As mentioned in the "Software Organization" section above, we may want to implement the reverse mode of autodifferentiation in the future. This could be a useful addition to our package, because, as mentioned above the reverse mode is more efficient when the number of independent variables is much greater than the number of functions. We would likely create reverse mode as a separate module within our autodiff directory.

Several changes should be performed:

* Because reverse mode includes a forward pass and a reverse calculation, we may record the graph structrure when we create the final function. In other words, when we create a new variable using a function (or variable operators) from 1 or 2 base variables, the function should record those base variables, and store them is a list or dict for later use.
* The definition of gradient for the Variable class could be optimized. Because we only need to record at most 2 gradients (one for each child base variable), we can use a tuple or just use two attributes to record the gradient within the Variable class.

# Future